In [ ]:
!pip install nltk

In [53]:
!pip install lightgbm sentence-transformers


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 325.1 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.5 MB 651.6 kB/s eta 0:00:03
   ------- -------------------------------- 0.3/1.5 MB 1.7 MB/s eta 0:00:01
   -------------- ------------------------- 0.5/1.5 MB 2.4 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 2.6 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.5 MB 3.2 MB/s eta 0:00:01
   -------------------------------------- - 1.4/1.5 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install scikit-learn

In [ ]:
pip install -U sentence-transformers


# Dataset Initializaiton

In [48]:
import pandas as pd
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

SyntaxError: invalid syntax (419723173.py, line 3)

In [50]:

print(train_df['BTL '].value_counts())

BTL 
1    171
6    160
5    149
3    138
4    119
2     86
Name: count, dtype: int64


In [6]:
print(len(train_df))
print(len(test_df))

823
206


In [45]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Load your train/test data
X_train = train_df['QUESTION'].tolist()
y_train = train_df['BTL '].tolist()

X_test = test_df['QUESTION'].tolist()
y_test = test_df['BTL '].tolist()

# Load lightweight SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Get embeddings
X_train_emb = model.encode(X_train, batch_size=32, show_progress_bar=True)
X_test_emb = model.encode(X_test, batch_size=32, show_progress_bar=True)

# Train classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_emb, y_train)

# Predict & evaluate
y_pred = clf.predict(X_test_emb)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


c:\Users\Welcome\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Welcome\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Welcome\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer 

Accuracy: 0.6504854368932039

Classification Report:
               precision    recall  f1-score   support

           1       0.74      0.65      0.69        43
           2       0.71      0.55      0.62        22
           3       0.44      0.42      0.43        38
           4       0.72      0.67      0.69        39
           5       0.51      0.65      0.57        31
           6       0.80      0.97      0.88        33

    accuracy                           0.65       206
   macro avg       0.65      0.65      0.65       206
weighted avg       0.65      0.65      0.65       206



In [55]:
from sentence_transformers import SentenceTransformer
from lightgbm import LGBMClassifier

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')  # smaller and better than paraphrase-MiniLM

X_train = model.encode(train_df['QUESTION'].tolist(), show_progress_bar=True)
y_train = train_df['BTL '].tolist()

X_test = model.encode(test_df['QUESTION'].tolist(), show_progress_bar=True)
y_test = test_df['BTL '].tolist()

clf = LGBMClassifier(n_estimators=200)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Batches: 100%|██████████| 7/7 [00:02<00:00,  2.51it/s]


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005911 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97920
[LightGBM] [Info] Number of data points in the train set: 823, number of used features: 384
[LightGBM] [Info] Start training from score -1.571293
[LightGBM] [Info] Start training from score -2.258609
[LightGBM] [Info] Start training from score -1.785703
[LightGBM] [Info] Start training from score -1.933833
[LightGBM] [Info] Start training from score -1.709010
[LightGBM] [Info] Start training from score -1.637782
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

c:\Users\Welcome\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


# SVM

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import make_pipeline

In [56]:

vectorizer = TfidfVectorizer(
    ngram_range=(1, 3),              # Unigram to Trigram
    stop_words='english',
    max_features=20000,
    sublinear_tf=True,               # log scale term frequency
    norm='l2'                        # normalize vectors
)
svm_model = SVC(kernel='linear', random_state=42)

pipeline = make_pipeline(vectorizer, svm_model)

pipeline.fit(train_df['QUESTION'], train_df['BTL '])

predictions = pipeline.predict(test_df['QUESTION'])

print("Accuracy:", accuracy_score(test_df['BTL '], predictions))
print("\nClassification Report:\n", classification_report(test_df['BTL '], predictions))


Accuracy: 0.7572815533980582

Classification Report:
               precision    recall  f1-score   support

           1       0.67      0.91      0.77        43
           2       0.80      0.55      0.65        22
           3       0.75      0.55      0.64        38
           4       0.86      0.77      0.81        39
           5       0.76      0.71      0.73        31
           6       0.78      0.97      0.86        33

    accuracy                           0.76       206
   macro avg       0.77      0.74      0.74       206
weighted avg       0.77      0.76      0.75       206



In [57]:
import joblib
joblib.dump(pipeline, "btl_model.pkl")


['btl_model.pkl']

In [40]:
from sklearn.ensemble import RandomForestClassifier

pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 2), stop_words='english'),
    RandomForestClassifier(n_estimators=200, random_state=42)
)


pipeline.fit(train_df['QUESTION'], train_df['BTL '])

predictions = pipeline.predict(test_df['QUESTION'])

print("Accuracy:", accuracy_score(test_df['BTL '], predictions))
print("\nClassification Report:\n", classification_report(test_df['BTL '], predictions))

Accuracy: 0.6796116504854369

Classification Report:
               precision    recall  f1-score   support

           1       0.52      0.93      0.67        43
           2       0.57      0.55      0.56        22
           3       0.56      0.26      0.36        38
           4       0.82      0.69      0.75        39
           5       0.90      0.61      0.73        31
           6       0.89      0.97      0.93        33

    accuracy                           0.68       206
   macro avg       0.71      0.67      0.67       206
weighted avg       0.71      0.68      0.67       206



# BERT

In [9]:
!pip install transformers datasets torch


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
from transformers import DataCollatorWithPadding

c:\Users\Welcome\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Convert to Hugging Face Dataset format
train_dataset = load_dataset('csv', data_files='train_df.csv')
test_dataset = load_dataset('csv', data_files='test_df.csv')

train_dataset = train_dataset['train']
test_dataset = test_dataset['train']

In [3]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['QUESTION'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format to torch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'BTL '])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'BTL '])


In [4]:
num_labels = 6  # For BTL levels 1 to 6
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


c:\Users\Welcome\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:
def predict_btl(question):
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item() + 1  # Adding 1 to match BTL range 1-6
    return predicted_label

# Example
question = "What is the time complexity of quicksort?"
predicted_btl = predict_btl(question)
print(f"Predicted BTL Level: {predicted_btl}")
